In [1]:
# This notebook includes model developement with target variable as Stress_Level

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [3]:
file_path = 'Data/Cleaned_Impact_of_Remote_Work_on_Mental_Health.csv'
df = pd.read_csv(file_path)

In [4]:
df['Mental_Health_Condition'].value_counts()

Mental_Health_Condition
Burnout       1280
Anxiety       1278
Depression    1246
Unknown       1196
Name: count, dtype: int64

#### Encoding

In [5]:
df['Satisfaction_with_Remote_Work_Encoded'] = df['Satisfaction_with_Remote_Work'].apply(lambda x: 1 if x == 'Satisfied' else 0)

In [6]:
df['Sleep_Quality_Encoded'] = df['Sleep_Quality'].apply(lambda x: 2 if x == 'Good' else (1 if x == 'Average' else 0))

In [7]:
def classify_stress(row):
    if row['Stress_Level'] == 'High':
        return 1
    elif row['Stress_Level'] == 'Medium' and (row['Mental_Health_Condition'] != 'Unknown' or row['Sleep_Quality_Encoded'] == 0):
        return 1
    elif row['Stress_Level'] == 'Low' and (row['Mental_Health_Condition'] != 'Unknown' or row['Sleep_Quality_Encoded'] == 0):
        return 1
    else:
        return 0

df['Stress_Level_Binary'] = df.apply(classify_stress, axis = 1)

In [8]:
features_to_drop = ['Stress_Level', 'Employee_ID', 'Satisfaction_with_Remote_Work', 'Sleep_Quality', 'Region', 'Gender', 
                    'Job_Role', 'Stress_Level_Binary']

In [9]:
X = pd.get_dummies(df.drop(features_to_drop, axis = 1), drop_first = True)
y = df['Stress_Level_Binary']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [11]:
class_distribution = y_train.value_counts()
print(class_distribution)

Stress_Level_Binary
1    3584
0     416
Name: count, dtype: int64


In [12]:
scale_pos_weight = class_distribution.max() / class_distribution.min()
scale_pos_weight

np.float64(8.615384615384615)

## Using XGBoost

#### Tuning

In [13]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
import numpy as np

def XGB_Bayesian(max_depth, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda):
    model = XGBClassifier(
        max_depth = int(max_depth),
        learning_rate = learning_rate,
        n_estimators = int(n_estimators),
        subsample = subsample,
        colsample_bytree = colsample_bytree,
        reg_alpha = reg_alpha,
        reg_lambda = reg_lambda,
        scale_pos_weight = scale_pos_weight,
        random_state = 42
    )
    
    # Performing cross-validation
    cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
    scores = cross_val_score(model, X_train, y_train, cv = cv, scoring = 'f1')
    return np.mean(scores)

param_bounds = {
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.1),
    'n_estimators': (50, 300),
    'subsample': (0.4, 1.0),
    'reg_alpha': (0.0, 1.0),
    'reg_lambda': (0.0, 1.0),
    'colsample_bytree': (0.6, 1.0)
}

optimizer = BayesianOptimization(
    f = XGB_Bayesian,
    pbounds = param_bounds,
    random_state = 42,
    verbose = 2
)

optimizer.maximize(init_points = 20, n_iter = 50)
print("Best Parameters:", optimizer.max)

|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
| 1         | 0.9555    | 0.7498    | 0.09556   | 8.124     | 199.7     | 0.156     | 0.156     | 0.4349    |
| 2         | 0.9478    | 0.9465    | 0.0641    | 7.957     | 55.15     | 0.9699    | 0.8324    | 0.5274    |
| 3         | 0.9448    | 0.6727    | 0.02651   | 5.13      | 181.2     | 0.4319    | 0.2912    | 0.7671    |
| 4         | 0.9445    | 0.6558    | 0.03629   | 5.565     | 164.0     | 0.7852    | 0.1997    | 0.7085    |
| 5         | 0.945     | 0.837     | 0.01418   | 7.253     | 92.63     | 0.06505   | 0.9489    | 0.9794    |
| 6         | 0.9449    | 0.9234    | 0.03742   | 3.684     | 221.1     | 0.4402    | 0.122     | 0.6971    |
| 7         | 0.9508    | 0.6138    | 0.09184   | 4.811     | 215.6     | 0.3117    | 0.5201    | 0.728     |
| 8       

#### Train

In [14]:
xgb_model = XGBClassifier(
    scale_pos_weight = scale_pos_weight,
    learning_rate = 0.09628777183080882,
    n_estimators = 269,
    max_depth = 7,
    colsample_bytree = 0.7488332077342038,
    subsample = 0.713566917499455,
    reg_alpha = 0.028435014918487433,
    reg_lambda = 0.1736203484849288,
    random_state = 42
)

xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7488332077342038, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.09628777183080882,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=269, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

#### Predict

In [15]:
y_pred = xgb_model.predict(X_test)

print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.55      0.58        98
           1       0.95      0.96      0.96       902

    accuracy                           0.92      1000
   macro avg       0.78      0.76      0.77      1000
weighted avg       0.92      0.92      0.92      1000

Confusion Matrix:
 [[ 54  44]
 [ 35 867]]


#### Change threshold to improve performance

In [16]:
from sklearn.metrics import classification_report, precision_recall_curve

y_probs = xgb_model.predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, y_probs)
f1_scores = 2 * (precision * recall) / (precision + recall)
best_threshold = thresholds[np.argmax(f1_scores)]

y_pred_adjusted = (y_probs >= best_threshold).astype(int)
print("\nClassification Report:\n", classification_report(y_test, y_pred_adjusted))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_adjusted))


Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.96      0.77        98
           1       1.00      0.94      0.97       902

    accuracy                           0.94      1000
   macro avg       0.82      0.95      0.87      1000
weighted avg       0.96      0.94      0.95      1000

Confusion Matrix:
 [[ 94   4]
 [ 51 851]]


## Using RandomForest

#### Tuning

In [28]:
from sklearn.ensemble import RandomForestClassifier

def RandomForest_Bayesian(n_estimators, max_depth, min_samples_split, min_samples_leaf):
    model = RandomForestClassifier(
        max_depth = int(max_depth),
        n_estimators = int(n_estimators),
        min_samples_split = int(min_samples_split),
        min_samples_leaf = int(min_samples_leaf),
        class_weight = 'balanced',
        random_state = 42
    )
    
    # Performing cross-validation
    cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
    scores = cross_val_score(model, X_train, y_train, cv = cv, scoring = 'f1')
    return np.mean(scores)

param_bounds = {
    'n_estimators': (50, 300),
    'max_depth': (3, 20),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 5)
}

optimizer = BayesianOptimization(
    f = RandomForest_Bayesian,
    pbounds = param_bounds,
    random_state = 42,
    verbose = 2
)

optimizer.maximize(init_points = 20, n_iter = 50)
print("Best Parameters:", optimizer.max)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.9684    | 9.367     | 4.803     | 7.856     | 199.7     |
| 2         | 0.9684    | 5.652     | 1.624     | 2.465     | 266.5     |
| 3         | 0.9684    | 13.22     | 3.832     | 2.165     | 292.5     |
| 4         | 0.9648    | 17.15     | 1.849     | 3.455     | 95.85     |
| 5         | 0.9684    | 8.172     | 3.099     | 5.456     | 122.8     |
| 6         | 0.9682    | 13.4      | 1.558     | 4.337     | 141.6     |
| 7         | 0.9684    | 10.75     | 4.141     | 3.597     | 178.6     |
| 8         | 0.9681    | 13.07     | 1.186     | 6.86      | 92.63     |
| 9         | 0.9684    | 4.106     | 4.796     | 9.725     | 252.1     |
| 10        | 0.9683    | 8.178     | 1.391     | 7.474     | 160.0     |
| 11        | 0.9684    | 5.075     | 2.981     | 2.275     | 277.3     |
| 12        | 0.9684    | 7.399     | 

#### Train

In [29]:
rf_model = RandomForestClassifier(
    n_estimators = 273,
    max_depth = 17,
    min_samples_split = 9,
    min_samples_leaf = 1,
    class_weight = 'balanced',
    random_state = 42,
)

rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=17,
                       min_samples_split=9, n_estimators=273, random_state=42)

#### Predict

In [32]:
y_pred2 = rf_model.predict(X_test)

print("\nClassification Report:\n", classification_report(y_test, y_pred2))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred2))


Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.98      0.77        98
           1       1.00      0.94      0.97       902

    accuracy                           0.94      1000
   macro avg       0.82      0.96      0.87      1000
weighted avg       0.96      0.94      0.95      1000

Confusion Matrix:
 [[ 96   2]
 [ 55 847]]


#### Change threshold to improve performance

In [33]:
y_probs2 = rf_model.predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, y_probs)
f1_scores = 2 * (precision * recall) / (precision + recall)
best_threshold = thresholds[np.argmax(f1_scores)]

y_pred_adjusted2 = (y_probs >= best_threshold).astype(int)
print("\nClassification Report:\n", classification_report(y_test, y_pred_adjusted2))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_adjusted2))


Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.96      0.77        98
           1       1.00      0.94      0.97       902

    accuracy                           0.94      1000
   macro avg       0.82      0.95      0.87      1000
weighted avg       0.96      0.94      0.95      1000

Confusion Matrix:
 [[ 94   4]
 [ 51 851]]
